In [1]:
import tensorflow as tf
import csv
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import imghdr
import IPython.display as display
import numpy as np
from pathlib import Path

In [2]:
def _int64_feature(value):
#Wrapper for inserting int64 features into Example proto.
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
#Wrapper for inserting bytes features into Example proto.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string)
    feature = {
      'height': _int64_feature(image_shape.shape[0]),
      'width': _int64_feature(image_shape.shape[1]),
      'depth': _int64_feature(image_shape.shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }
    return tf.train.Example(features=tf.train.Features(feature=feature))


record_file = 'test.tfrecords'
numbers = 10
buffer_size = 0
with tf.io.TFRecordWriter(record_file) as writer:
    for i in range(numbers):
        folder = i
        data_folder = Path("mnist data/trainingSample/trainingSample/%d" %folder)
        directory = os.fsencode(data_folder)
        for file in os.listdir(directory):
            filename = data_folder / os.fsdecode(file)
            image_string = open(filename,'rb').read()
            label = folder
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())
            buffer_size +=1
#for line in str(image_example(image_string, label)).split('\n')[:15]:
#    print(line)
#print('...')


In [3]:
raw_image_dataset = tf.data.TFRecordDataset('test.tfrecords')

def _parse_image_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    # Create a dictionary describing the features.
    image_feature_description = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    image_features = tf.io.parse_single_example(example_proto,image_feature_description)
    image_buffer = image_features['image_raw']
    image = tf.image.decode_jpeg(image_buffer,channels = 1)
    image = tf.image.convert_image_dtype(image,dtype=tf.float32)*(1. / 255)
    image_shape = tf.stack([28,28,1])
    image = tf.reshape(image,image_shape)
    label = tf.cast(image_features['label'],tf.uint64)
    label = tf.squeeze(label)
    return image,label

raw_image_dataset = raw_image_dataset.shuffle(1000)
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset = parsed_image_dataset.batch(64)
parsed_image_dataset = parsed_image_dataset.prefetch(1)

In [4]:
for raw_record in raw_image_dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "depth"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "height"
    value {
      int64_list {
        value: 28
      }
    }
  }
  feature {
    key: "image_raw"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\300\000\013\010\000\034\000\034\001\001\021\000\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\21

In [12]:
for image, label in parsed_image_dataset.take(1):
    print(image.shape, label.shape)

(64, 28, 28, 1) (64,)


In [6]:
#creating the model
model = tf.keras.Sequential()
model.add(Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(3,3), padding='same'))
model.add(Flatten(input_shape=(28,28,1)))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

In [7]:
model.compile(optimizer='adam', metrics=['sparse_categorical_accuracy'], loss='sparse_categorical_crossentropy')

In [13]:
model.fit(parsed_image_dataset, epochs=1)

      1/Unknown - 0s 63ms/step

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-8-93a1cbd10f21>:1) ]] [Op:__inference_distributed_function_10574]

Function call stack:
distributed_function
